Stand: in Entwicklung
TODO
- die Api schnittstellen fehlen
- Datenaufbereitung
- Symbole hochladen

In [1]:
import sys
import os
import pandas as pd
from datetime import datetime

# Pfad zum Modul: /home/ageq/Git_Projects/MLdatalake/customs
current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', 'customs')))

from Module.SQLAlchemy_functions import start_session, Symbol, MinuteBar, FiveMinuteBar , ThirtyMinuteBar

#/home/ageq/Git_Projects/MLdatalake/customs/Module/Grabber/Yahoofinance.py

from Module.Grabber.Yahoofinance import load_yf_data

# CSV-Datei Pfad
csv_file_path = 'ref_assets.csv'

#session = start_session()

In [2]:

def check_if_asset_exists(session, row):
    # Prüfe, ob das Asset in der Datenbank existiert
    asset = session.query(Symbol).filter_by(ticker=row['ticker']).first()

    if asset:
        print(f"Asset {row['ticker']} existiert bereits. Überprüfe das letzte Update.")
        
        # Letztes Update des Assets prüfen und aktualisieren
        last_update = update_asset_last_update(session, row)
        
        if last_update:
            print("Vorheriges last_update in row:", row['last_update'])
            # Aktualisiere `row['last_update']` mit dem neuen Wert
            row['last_update'] = last_update
            print("Neues last_update in row:", row['last_update'])
            
            try:
                # Speichern in der Datenbank, wenn nötig
                asset.last_update = last_update  # Setze das letzte Update-Datum im Asset
                session.commit()  # Änderungen in der Datenbank speichern
                print(f"Asset {row['ticker']} erfolgreich aktualisiert mit letztem Update: {last_update}.")
            except Exception as e:
                session.rollback()  # Bei Fehlern Rollback durchführen
                print(f"Fehler beim Speichern des Updates für Asset {row['ticker']}: {e}")
        else:
            print(f"Kein Update für {row['ticker']} erforderlich.")
    else:
        print(f"Asset {row['ticker']} existiert nicht, füge es hinzu.")
        asset = add_new_asset(session, row)
    
    # Gib die aktualisierte `row` zurück
    return asset, row

"""
# Neues Asset hinzufügen
def add_new_asset(session, row):
    try:
        asset = Symbol(
            ticker=row['ticker'],
            name=row['name'],
            market=row['market'],
            active=row['active']
        )
        session.add(asset)
        session.commit()
        print(f"Asset {row['ticker']} erfolgreich hinzugefügt.")
    except Exception as e:
        print(f"Fehler beim Hinzufügen des Assets {row['ticker']}: {e}")
"""       
def add_new_asset(session, row):
    try:
        # Erstelle ein neues Asset mit den gegebenen Informationen
        asset = Symbol(
            ticker=row['ticker'],
            name=row['name'],
            market=row['market'],
            active=row['active'],
            start_date=datetime.now()  # Setze das Startdatum auf das aktuelle Datum
        )
        session.add(asset)
        session.commit()  # Sicherstellen, dass das Asset hinzugefügt wurde
        print(f"Asset {row['ticker']} erfolgreich hinzugefügt.")

        # Aktualisiere die CSV-Datei mit dem neuen Startdatum
        update_csv_start_date(csv_file_path, row['ticker'], asset.start_date)
        
        return asset
    except Exception as e:
        session.rollback()
        print(f"Fehler beim Hinzufügen des Assets {row['ticker']}: {e}")
        return None


# Aktualisieren der letzten Daten in der Asset-Tabelle
def update_asset_last_update(session, row):
    try:
        # Abfrage des Assets anhand des Tickers, um sicherzustellen, dass die Symbol-ID stimmt
        asset = session.query(Symbol).filter_by(ticker=row['ticker']).first()
        print(f"Symbol-ID für {row['ticker']}: {asset.id}")
        
        # Überprüfen, welcher Zeitrahmen vorliegt
        if row['timeframe'] == '5m':
            last_update = session.query(FiveMinuteBar).filter_by(symbol_id=asset.id).order_by(FiveMinuteBar.date.desc()).first()

        # Falls ein Update existiert, Datum zurückgeben
        if last_update:
            print(f"Letztes Update für {row['ticker']} am {last_update.date}")
            return last_update.date
        else:
            print(f"Kein letzter Eintrag für {row['ticker']} im Zeitrahmen {row['timeframe']} gefunden.")
            return None
    except Exception as e:
        session.rollback()
        print(f"Fehler beim Abrufen des letzten Updates für {row['ticker']}: {e}")
        return None




# Daten in MinuteBar-Tabelle einfügen
# und in die Datenbank einfügen
def insert_minute_bar(session, data, symbol_id):
    try:
        for index, row in data.iterrows():
            minute_bar = MinuteBar(
                date=row['date'],
                open=row['open'],
                high=row['high'],
                low=row['low'],
                close=row['close'],
                volume=row['volume'],
                symbol_id=symbol_id
            )
            session.add(minute_bar)
        session.commit()
        print(f"Minutenbalken für Symbol-ID {symbol_id} erfolgreich eingefügt.")
        
    except Exception as e:
        print(f"Fehler beim Einfügen der MinuteBars für Symbol-ID {symbol_id}: {e}")
        
# Daten für 5-Minuten und 30-Minuten-Bars sind ähnlich:
# Daten in FiveMinuteBar-Tabelle einfügen
def insert_five_minute_bar(session, data, symbol_id):
    try:
        for index, row in data.iterrows():
            # Prüfen, ob der Eintrag bereits existiert
            existing_entry = session.query(FiveMinuteBar).filter_by(symbol_id=symbol_id, date=row['date']).first()

            if existing_entry:
                print(f"Eintrag für Symbol-ID {symbol_id} am {row['date']} existiert bereits, wird übersprungen.")
                continue  # Überspringen, falls Eintrag vorhanden
            
            # Wenn kein Eintrag vorhanden ist, neuen Eintrag hinzufügen
            five_minute_bar = FiveMinuteBar(
                date=row['date'],
                open=row['open'],
                high=row['high'],
                low=row['low'],
                close=row['close'],
                volume=row['volume'],
                symbol_id=symbol_id
            )
            session.add(five_minute_bar)

        session.commit()
        print(f"5-Minuten-Balken für Symbol-ID {symbol_id} erfolgreich eingefügt.")

    except Exception as e:
        session.rollback()  # Session zurücksetzen
        print(f"Fehler beim Einfügen der 5-Minuten-Bars für Symbol-ID {symbol_id}: {e}")

# insert_five_minute_bar, insert_thirty_minute_bar (folgt ähnlicher Logik)


def load_assets_from_csv(file_path, session):
    try:
        assets_df = pd.read_csv(file_path)
        with open(file_path, 'r') as file:
            content = file.read()
            if not content.strip():
                print("Die CSV-Datei ist leer.")
                return
    except Exception as e:
        print(f"Fehler beim Einlesen der CSV-Datei: {e}")
        return
    
    for index, row in assets_df.iterrows():
        try:
            process_asset(session, row)
        except Exception as e:
            print(f"Fehler beim Verarbeiten des Assets {row['ticker']}: {e}")
            continue  # Fortfahren mit dem nächsten Asset

def update_csv_start_date(file_path, ticker, start_date):
    try:
        # Lesen der CSV-Datei
        df = pd.read_csv(file_path)
        
        # Aktualisieren des Startdatums für das Asset
        df.loc[df['ticker'] == ticker, 'start_date'] = start_date.strftime('%Y-%m-%d')
        
        # Speichern der aktualisierten CSV-Datei
        df.to_csv(file_path, index=False)
        print(f"Startdatum für Asset {ticker} in der CSV-Datei aktualisiert.")
    except Exception as e:
        print(f"Fehler beim Aktualisieren der CSV-Datei: {e}")
        
def update_csv_last_update(session, row, csv_file_path):
    last_update = update_asset_last_update(session, row)
    if last_update:
        # Lade die CSV-Datei
        df = pd.read_csv(csv_file_path)
        
        # Finde den entsprechenden Eintrag in der CSV und aktualisiere das Last-Update-Feld
        df.loc[df['ticker'] == row['ticker'], 'last_update'] = last_update
        
        # Schreibe die CSV-Datei zurück
        df.to_csv(csv_file_path, index=False)
        print(f"CSV-Datei für {row['ticker']} aktualisiert mit neuem Last Update: {last_update}")
    else:
        print(f"Kein Last Update für {row['ticker']} gefunden, CSV wurde nicht aktualisiert.")
        
        


# Dummy-Funktionen für verschiedene Exchanges
def process_binance_asset(session, row):
    print("Binance ist noch nicht implementiert.")
    #print(f"Verarbeite Binance Asset: {row['ticker']}")

def process_yahoof_asset(session, row):

    #prüfe ob das Asset existiert
    #print("row: ", row)
    asset , row= check_if_asset_exists(session, row)
    print("updatet row: ", row)
    
    data = load_yf_data(row)
    print(f"Verarbeite Yahoo Finance Asset: {row['ticker']}")
    if row['timeframe'] == '1m':
        insert_minute_bar(session, data, asset.id)
    elif row['timeframe'] == '5m':
        insert_five_minute_bar(session, data, asset.id)
    else:
        print(f"Zeitrahmen {row['timeframe']} wird nicht unterstützt.")
        return
    update_csv_last_update(session, row, csv_file_path)
    
exchange_function_map = {
    'Binance': process_binance_asset,
    'Yahoo Finance': process_yahoof_asset
}


# Asset-Verarbeitung (Erstellen oder Aktualisieren)
def process_asset(session, row):

    # Verarbeiten des Assets basierend auf dem Exchange
    exchange = row['exchange']
    if exchange in exchange_function_map:
        exchange_function_map[exchange](session, row)
    else:
        print(f"Unbekannter Exchange: {exchange}")


In [3]:
# Daten in MinuteBar-Tabelle einfügen
# und in die Datenbank einfügen
def insert_minute_bar(session, data, symbol_id):
    try:
        for index, row in data.iterrows():
            minute_bar = MinuteBar(
                date=row['date'],
                open=row['open'],
                high=row['high'],
                low=row['low'],
                close=row['close'],
                volume=row['volume'],
                symbol_id=symbol_id
            )
            session.add(minute_bar)
        session.commit()
        print(f"Minutenbalken für Symbol-ID {symbol_id} erfolgreich eingefügt.")
        
    except Exception as e:
        print(f"Fehler beim Einfügen der MinuteBars für Symbol-ID {symbol_id}: {e}")
        
# Daten für 5-Minuten und 30-Minuten-Bars sind ähnlich:
# Daten in FiveMinuteBar-Tabelle einfügen
def insert_five_minute_bar(session, data, symbol_id):
    try:
        for index, row in data.iterrows():
            # Prüfen, ob der Eintrag bereits existiert
            existing_entry = session.query(FiveMinuteBar).filter_by(symbol_id=symbol_id, date=row['date']).first()

            if existing_entry:
                print(f"Eintrag für Symbol-ID {symbol_id} am {row['date']} existiert bereits, wird übersprungen.")
                continue  # Überspringen, falls Eintrag vorhanden
            
            # Wenn kein Eintrag vorhanden ist, neuen Eintrag hinzufügen
            five_minute_bar = FiveMinuteBar(
                date=row['date'],
                open=row['open'],
                high=row['high'],
                low=row['low'],
                close=row['close'],
                volume=row['volume'],
                symbol_id=symbol_id
            )
            session.add(five_minute_bar)

        session.commit()
        print(f"5-Minuten-Balken für Symbol-ID {symbol_id} erfolgreich eingefügt.")

    except Exception as e:
        session.rollback()  # Session zurücksetzen
        print(f"Fehler beim Einfügen der 5-Minuten-Bars für Symbol-ID {symbol_id}: {e}")

# insert_five_minute_bar, insert_thirty_minute_bar (folgt ähnlicher Logik)


In [4]:
def main():
    # Datenbank-Verbindung und Session erstellen

    
    # CSV-Datei Pfad
    csv_file_path = 'ref_assets.csv'
    
    # Tabellen in der Datenbank erstellen
    #create_tables(engine)
    session= start_session()
    # Assets aus CSV laden und verarbeiten
    load_assets_from_csv(csv_file_path, session)

if __name__ == "__main__":
    main()



session started
Binance ist noch nicht implementiert.
row:  ticker                  AAPL
name                   Apple
market                 stock
exchange       Yahoo Finance
start_date        2024-10-22
last_update       2024-01-22
timeframe                 5m
active                     1
Name: 1, dtype: object
Asset AAPL existiert bereits. Überprüfe das letzte Update.
Symbol-ID für AAPL: 351
Letztes Update für AAPL am 2024-10-22 19:55:00
Vorheriges last_update in row: 2024-01-22
Neues last_update in row: 2024-10-22 19:55:00
Asset AAPL erfolgreich aktualisiert mit letztem Update: 2024-10-22 19:55:00.
updatet row:  ticker                        AAPL
name                         Apple
market                       stock
exchange             Yahoo Finance
start_date              2024-10-22
last_update    2024-10-22 19:55:00
timeframe                       5m
active                           1
Name: 1, dtype: object
Fehler beim Verarbeiten des Assets AAPL: 'datetime.datetime' object has n